# EDA & VISUALISATION

In [2]:
import pandas as pd
import numpy as np
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from sklearn.preprocessing import LabelEncoder
import time

In [16]:
df = pd.read_csv("C:\\Users\\amitx\\Desktop\\עמית קבצים\\לימודים\\asos data.csv")
df

,price,color,likes,brand,material type,Percentage discount,Rate,Fit,Saling fast,in sale,category,sex
0,370.00 ILS,BROWN,NaN,NaN,NaN,0,NaN,NaN,SELLING FAST,0,hoodies_sweatshirts,Women
1,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,hoodies_sweatshirts,Women
2,280.00 ILS,Cream,NaN,The North Face,NaN,0,NaN,NaN,NaN,0,hoodies_sweatshirts,Women
3,450.00 ILS,BLACK,1K,adidas Originals,"Fleece: soft, cosy and insulating\n\nMain: 100...",0,NaN,Relaxed fit,NaN,0,hoodies_sweatshirts,Women
4,280.00 ILS,Charcoal,6K,NaN,NaN,0,NaN,Relaxed fit,NaN,0,hoodies_sweatshirts,Women
...,...,...,...,...,...,...,...,...,...,...,...,...
32251,195.00 ILS,Navy,NaN,NaN,NaN,0,NaN,NaN,NaN,0,T-shirts & Vests,Men
32252,185.00 ILS,Off noir,NaN,NaN,NaN,0,NaN,NaN,NaN,0,T-shirts & Vests,Men
32253,225.00 ILS,Black,NaN,NaN,NaN,0,NaN,NaN,NaN,0,T-shirts & Vests,Men
32254,195.00 ILS,White,NaN,NaN,NaN,0,NaN,NaN,NaN,0,T-shirts & Vests,Men


Load dataset

In [4]:
df = pd.read_csv("C:\\Users\\amitx\\Desktop\\עמית קבצים\\לימודים\\שנה ב\\מדעי הנתנוים\\פרויקט\\פרויקט מתוקן\\Data no clean.csv")
df

,price,color,likes,brand,material type,Percentage discount,Rate,Fit,Saling fast,in sale,category,sex
0,Now 236.01 ILS,RED,363,NaN,"Fleece: soft, cosy and insulating\n\nMain: 100...",(-40%),NaN,Regular fit,NaN,1,Hoodies & Sweatshirts,Women
1,Now 270.28 ILS,BLACK,1K,NaN,Sweatshirt fabric: soft and warm\n\nMain: 100%...,(-29%),4.3,Oversized fit,NaN,1,Hoodies & Sweatshirts,Women
2,Now 165.32 ILS,BLACK,132,NaN,NaN,(-29%),NaN,Regular fit,NaN,1,Hoodies & Sweatshirts,Women
3,Now 238.79 ILS,BLACK,390,NaN,NaN,(-29%),NaN,Regular fit,NaN,1,Hoodies & Sweatshirts,Women
4,Now 217.80 ILS,BLUE,712,NaN,Sweatshirt fabric: soft and warm\n\nMain: 80% ...,(-24%),4.5,Oversized fit,NaN,1,Hoodies & Sweatshirts,Women
...,...,...,...,...,...,...,...,...,...,...,...,...
18062,195.00 ILS,Navy,NaN,NaN,NaN,0,NaN,NaN,NaN,0,T-shirts & Vests,Men
18063,185.00 ILS,Off noir,NaN,NaN,NaN,0,NaN,NaN,NaN,0,T-shirts & Vests,Men
18064,225.00 ILS,Black,NaN,NaN,NaN,0,NaN,NaN,NaN,0,T-shirts & Vests,Men
18065,195.00 ILS,White,NaN,NaN,NaN,0,NaN,NaN,NaN,0,T-shirts & Vests,Men


In the following cell, we wrote a function that swiches capital letters to lower case letters to unify the data and reduce duplicates. This function also changes colums to category data type. 

In [5]:
def clean_df(df, columns_to_clean):
    # Convert specified columns to lowercase
    df[columns_to_clean] = df[columns_to_clean].apply(lambda x: x.str.lower())
    
    # Convert specified columns to category data type
    for col in columns_to_clean:
        df[col] = df[col].astype('category')
    
    return df

This function codes categories to numbers and adds a column with the values of the numbers. 

In [6]:
def label_encode_columns(df, cols):
    le = LabelEncoder()
    for col in cols:
        df[col+'_encoded'] = le.fit_transform(df.loc[:, col])
    return df

The fabric type descibes the type of material (cotton, wool, polyester, etc...) and sometimes the type of texture (jersey, stretch, jeans, etc...) and sometimes many combinations in different percentages. Many types of textures showed up once without repeats therefore we converted to type of fabric or type of texture that is similar. We created an array with key words type of texture and type of fabric and in this way we cleaned the data that didn't repeat itself. 

In [7]:
def extract_fabric_types(df, arr):
    df['fabric types'] = df['material type'].apply(lambda x: next((t for t in x.split() if t in arr), None))
    df['fabric types'] = df['fabric types'].apply(lambda x: x.split()[0] if isinstance(x, str) else None)
    return df

Number of likes that were less an one thousand were documented by Asos normally as a number, but number of likes that ranged from one thousand to one million Asos documented multiplications of a thousand with the character 'k'. If the number was greater than one million it was written as a multiplication of a million with the character 'm'. In order to receive the number of likes separately we removed the letter and multiplied the number the the value of the letter. 

In [8]:
def str_to_float_likes(likes_str):
    if isinstance(likes_str, float):
        return likes_str
    likes_str = likes_str.replace(",", "")
    if likes_str.endswith("K"):
        return float(likes_str[:-1]) * 1000
    elif likes_str.endswith("M"):
        return float(likes_str[:-1]) * 1000000
    else:
        return float(likes_str)

The percentage of discount was documented as a string that held a number and ended in '%'. In the following function we removed the '%' charecter and the converted it to a whole number. 

In [9]:
def str_to_int_percentage(s):
    if isinstance(s, str):
        return int(s.strip('()').replace('%', '')) * -1
    else:
        return s

The following cell holds the conversion of a string to a number, the conversion of empty strings to the value NAN, cleaning of colors that appear less than one hunderd times, cleaning brands that appear less than fifty times, coding columns of strings to their numerical value and the calling of a function that was detailed above. 

In [10]:
df['price'] = df['price'].apply(lambda x: re.sub('[^0-9\.]', '', str(x)))
df['price'] = df['price'].replace('', np.nan)
df['price'] = df['price'].astype(float)
df=clean_df(df,["color","brand","material type","Fit","category","sex"])
df = df[df['color'].map(df['color'].value_counts()) > 100]
df = label_encode_columns(df, ['color'])
df = df[df['brand'].map(df['brand'].value_counts()) > 50]
df = label_encode_columns(df, ['brand'])
df["material type"] = df["material type"].apply(lambda x: x.lower() if isinstance(x, str) else x)
fabric_types = ['twill', 'fabric', 'jersey', 'shell', 'denim', 'woven', 'satin', 'stretch',
                    'velvet', 'fleece', 'corduroy', 'borg', 'flannel', 'faux leather', 'faux fur', 
                    'teddy', 'chiffon', 'ripstop', 'canvas', 'tricot', 'poplin']
df = extract_fabric_types(df,fabric_types)
df=clean_df(df,["fabric types"])
df = df.drop('material type', axis=1)
df['likes'] = df['likes'].apply(str_to_float_likes)
df['Percentage discount'] = df['Percentage discount'].apply(str_to_int_percentage)
df['Rate'] = df['Rate'].astype(float)
df = df[df['Fit'].map(df['Fit'].value_counts()) > 5]
df = label_encode_columns(df, ['Fit'])
df = label_encode_columns(df, ['category','sex','fabric types'])

C:\Users\amitx\AppData\Local\Temp\ipykernel_19252\417190844.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col+'_encoded'] = le.fit_transform(df.loc[:, col])


In [11]:
df

,price,color,likes,brand,Percentage discount,Rate,Fit,Saling fast,in sale,category,sex,color_encoded,brand_encoded,fabric types,Fit_encoded,category_encoded,sex_encoded,fabric types_encoded
6,158.60,brown,910.0,nike,45,NaN,oversized fit,NaN,1,hoodies & sweatshirts,women,3,16,fleece,12,1,1,4
27,196.80,black,216.0,under armour,29,NaN,regular fit,NaN,1,hoodies & sweatshirts,women,1,25,NaN,14,1,1,13
29,238.79,white,669.0,nike,29,NaN,relaxed fit,NaN,1,hoodies & sweatshirts,women,17,16,NaN,17,1,1,13
35,272.90,stone,887.0,nike,19,NaN,relaxed fit,NaN,1,hoodies & sweatshirts,women,16,16,NaN,17,1,1,13
36,314.89,black,640.0,nike,24,NaN,regular fit,NaN,1,hoodies & sweatshirts,women,1,16,NaN,14,1,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17807,150.00,brown,NaN,adidas,0,4.9,relaxed fit,NaN,0,t-shirts & vests,men,3,1,NaN,18,4,0,13
17820,105.00,white,702.0,nike,0,NaN,regular fit,NaN,0,t-shirts & vests,men,17,16,fabric,14,4,0,2
17915,155.00,green,NaN,the north face,0,NaN,regular fit,NaN,0,t-shirts & vests,men,6,22,NaN,14,4,0,13
17919,155.00,multi,NaN,asos design,0,NaN,muscle fit,NaN,0,t-shirts & vests,men,10,5,NaN,11,4,0,13


The following function cleans values and end values by EQR. 

In [12]:
def fix_outliers_iqr(df , col):
    Q25 = df[col].quantile(0.25)
    Q75 = df[col].quantile(0.75)

    iqr = Q75 - Q25
    outliers = df[(df[col] > Q25 - 1.5*iqr) & (df[col] < Q75 + 1.5*iqr)]
    return outliers

In [13]:
df = fix_outliers_iqr(df,"price")
df = fix_outliers_iqr(df,"likes")
df = df.drop_duplicates()
df.reset_index(inplace =True)
df['Rate'] = df['Rate'].fillna(0)
most_common = df['fabric types'].mode()[0]
df['fabric types'] = df['fabric types'].fillna(most_common)
df =df.drop("Saling fast",axis=1)
df

,index,price,color,likes,brand,Percentage discount,Rate,Fit,in sale,category,sex,color_encoded,brand_encoded,fabric types,Fit_encoded,category_encoded,sex_encoded,fabric types_encoded
0,6,158.60,brown,910.0,nike,45,0.0,oversized fit,1,hoodies & sweatshirts,women,3,16,fleece,12,1,1,4
1,27,196.80,black,216.0,under armour,29,0.0,regular fit,1,hoodies & sweatshirts,women,1,25,fabric,14,1,1,13
2,29,238.79,white,669.0,nike,29,0.0,relaxed fit,1,hoodies & sweatshirts,women,17,16,fabric,17,1,1,13
3,35,272.90,stone,887.0,nike,19,0.0,relaxed fit,1,hoodies & sweatshirts,women,16,16,fabric,17,1,1,13
4,36,314.89,black,640.0,nike,24,0.0,regular fit,1,hoodies & sweatshirts,women,1,16,fabric,14,1,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506,15133,225.00,black,118.0,napapijri,0,0.0,regular fit,0,t-shirts & vests,men,1,14,fabric,14,4,0,13
3507,15134,140.00,black,181.0,asos design,0,0.0,oversized fit,0,t-shirts & vests,men,1,5,fabric,12,4,0,13
3508,15135,225.00,black,125.0,napapijri,0,0.0,regular fit,0,t-shirts & vests,men,1,14,fabric,14,4,0,13
3509,16537,145.00,white,216.0,topman,0,0.0,oversized fit,0,t-shirts & vests,men,17,23,fabric,12,4,0,13


In [15]:
df

,price,color,likes,brand,Percentage discount,Rate,Fit,in sale,category,sex,color_encoded,brand_encoded,fabric types,Fit_encoded,category_encoded,sex_encoded,fabric types_encoded
0,158.60,brown,910.0,nike,45,0.0,oversized fit,1,hoodies & sweatshirts,women,3,16,fleece,12,1,1,4
1,196.80,black,216.0,under armour,29,0.0,regular fit,1,hoodies & sweatshirts,women,1,25,fabric,14,1,1,13
2,238.79,white,669.0,nike,29,0.0,relaxed fit,1,hoodies & sweatshirts,women,17,16,fabric,17,1,1,13
3,272.90,stone,887.0,nike,19,0.0,relaxed fit,1,hoodies & sweatshirts,women,16,16,fabric,17,1,1,13
4,314.89,black,640.0,nike,24,0.0,regular fit,1,hoodies & sweatshirts,women,1,16,fabric,14,1,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506,225.00,black,118.0,napapijri,0,0.0,regular fit,0,t-shirts & vests,men,1,14,fabric,14,4,0,13
3507,140.00,black,181.0,asos design,0,0.0,oversized fit,0,t-shirts & vests,men,1,5,fabric,12,4,0,13
3508,225.00,black,125.0,napapijri,0,0.0,regular fit,0,t-shirts & vests,men,1,14,fabric,14,4,0,13
3509,145.00,white,216.0,topman,0,0.0,oversized fit,0,t-shirts & vests,men,17,23,fabric,12,4,0,13
